# Genre song classification
In out database we have spectrograms and chomagrams from different song genres: Alternative, Classical, Dance, Pop, Rock and Techno

### Import needed libraries

In [1]:
# Import general purpose python libraries
import os
import matplotlib.pyplot as plt
from PIL import Image # For handling the images
import numpy as np
from tensorflow import keras

# Import different Keras functionalities
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Add, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import concatenate
from keras.constraints import MaxNorm
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import image_dataset_from_directory
from keras import backend as K


from keras.models import Model
from keras.layers import Input, Dense

from keras.models import load_model
import tensorflow as tf

from keras.applications import ResNet50

from pathlib import Path


# Import function to plot the results
!pip install seaborn
import plots

### Data Configuration Parameters
Configuration variables related to the data

In [2]:
# Randomize the initial network weights
random_seed = True

# Paths to where training, testing, and validation images are
database_dir = 'dataset/'
train_dir = 'dataset/training/spectrogram'
val_dir = 'dataset/val/spectrogram'
test_dir = 'dataset/test/spectrogram'

# Directory where to store weights of the model and results
root_dir = "results"
# Create root directory for results if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Output dimension (number of sublects in our problem)
num_classes = 6

# Name of each gesture of the database
CLASSES = [x for x in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, x))]
print(f'The classess to classify are: {CLASSES}')

# Parameters that characterize the images
img_height = 480
img_width = 640
img_channels = 3 # although some images could be rgb, we work with grayscale images
color_mode = 'rgb'

The classess to classify are: ['Alternative', 'Classical', 'Dance', 'Pop', 'Rock', 'Techno']


## Configuration Training Parameters & Loading of training, validation and test datasets

In [3]:
# Parameters that configures the training process
batch_size = 1  # Batch size
epochs = 30  # Number of epochs
initial_lr = 1e-4   # Learning rate
seed = 42  # Random number
num_layers = 'all'
modelCNN = 'ResNet'  # VGG, ResNet, Random_Model
version = f'BS{batch_size}_E{epochs}_LR{initial_lr}_Layers{num_layers}'
experiment_dir = f'{root_dir}/{modelCNN}'

# Create experiment directory if it does not exist
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# 1. Generate train dataset (ds) from directory of samples
train_ds = image_dataset_from_directory(directory=train_dir,
                                        label_mode = 'categorical',
                                        class_names=CLASSES,
                                        batch_size=batch_size,
                                        color_mode=color_mode,
                                        image_size=(img_width,img_height), shuffle=True)

# 2. Generate validation dataset (ds) from directory of samples
val_ds  = image_dataset_from_directory(directory=val_dir,
                                       label_mode = 'categorical',
                                       class_names=CLASSES,
                                       batch_size=batch_size,
                                       color_mode=color_mode,
                                       image_size=(img_width,img_height))

# 3. Generate test dataset (ds) from directory of samples
test_ds = image_dataset_from_directory(directory=test_dir,
                                       label_mode = 'categorical',
                                       class_names=CLASSES,
                                       batch_size=batch_size,
                                       color_mode=color_mode,
                                       image_size=(img_width,img_height),
                                       shuffle = False)

Found 1186 files belonging to 6 classes.
Found 204 files belonging to 6 classes.
Found 204 files belonging to 6 classes.


## Function def for plots

# Training process

## Available Models: VGG-16 & ResNet

### VGG-16

In [4]:
def VGG16(img_width,img_height,img_channels):
	#  dropout rate for FC layers
	dropout=0.5

	# CNN architecture
	input_image = Input(shape=(img_width,img_height,img_channels))
	x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(input_image)
	x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(x1)
	x1 = MaxPooling2D((2, 2))(x1)
	x1 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
	x1 = MaxPooling2D((2, 2))(x1)
	x1 = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(256, (1, 1),padding='same', activation='relu')(x1)
	x1 = MaxPooling2D((2, 2))(x1)
	x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
	x1 = MaxPooling2D((2, 2))(x1)
	x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
	x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
	x1 = MaxPooling2D((2, 2))(x1)

	x1 = Flatten()(x1)

	x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x1)
	x=Dropout(dropout)(x)
	x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x)
	x=Dropout(dropout)(x)
	out= Dense(num_classes, activation='softmax')(x)

	model = Model(inputs = input_image, outputs = out)

	return model

model = VGG16(img_width,img_height,img_channels)



### RestNet
This ResNet network it has been coded and not obtained from an already existing project. This ResNet structure has been chosen because its simplicity and, as seen along the curse, good performance.

In [5]:
def RestNet (img_width,img_height,img_channels):
  dropout=0.5 # Para evitar que haya overfitting cancela algunas conexiones entre las neuronas aleatoriamente.

  # CNN architecture
  input_image = Input(shape=(img_width,img_height,img_channels))
  x0 = Conv2D(64, (3, 3),padding='same', activation='relu')(input_image)
  x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(x0)
  x1 = MaxPooling2D((2, 2))(x1)
  x2 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
  x3 = Conv2D(128, (3, 3),padding='same')(x2)
  x4 = Add()([x3, x2])
  x4 = Activation('relu')(x4)
  x4 = MaxPooling2D((2, 2))(x4)
  x4 = Conv2D(256, (3, 3),padding='same', activation='relu')(x4)
  x5 = Conv2D(256, (3, 3),padding='same', activation='relu')(x4)
  x5 = Conv2D(256, (1, 1),padding='same')(x5)
  x6 = Add()([x5, x4])
  x6 = Activation('relu')(x6)
  x6 = MaxPooling2D((2, 2))(x6)
  x6 = Conv2D(512, (3, 3),padding='same', activation='relu')(x6)
  x7 = Conv2D(512, (3, 3),padding='same', activation='relu')(x6)
  x7 = Conv2D(512, (1, 1),padding='same')(x7)
  x8 = Add()([x7, x6])
  x8 = Activation('relu')(x8)
  x8 = MaxPooling2D((2, 2))(x8)
  x8 = Conv2D(512, (3, 3),padding='same', activation='relu')(x8)
  x9 = Conv2D(512, (3, 3),padding='same', activation='relu')(x8)
  x9 = Conv2D(512, (1, 1),padding='same')(x9)
  x10 = Add()([x9, x8])
  x10 = Activation('relu')(x10)
  x10 = MaxPooling2D((2, 2))(x10)
  x10 = Flatten()(x10)

  x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x10)
  x=Dropout(dropout)(x)
  x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x)
  x=Dropout(dropout)(x)
  out= Dense(num_classes, activation='softmax')(x)

  model = Model(inputs = input_image, outputs = out);

  return model

## Model execution

In [6]:
model = None
if modelCNN == 'VGG-16':
    model = VGG16(img_width,img_height,img_channels)
elif modelCNN == 'ResNet':
    model = RestNet (img_width,img_height,img_channels)
else:
    print('Wrong model selection or Model no available\n')

# Print the architecture of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 640, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 640, 480, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_14 (Conv2D)             (None, 640, 480, 64  36928       ['conv2d_13[0][0]']              
                                )                                                           

## Set model training process
#### Configuration of several training decisions:
1. Optimizer using `Adam`
2. Model training configuration using `compile` with `categorical_crossentropy` due to the classification labeling

In [7]:
# Set random seed
if random_seed:
    seed = np.random.randint(0,2*31-1)
else:
    seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)


# 1. Configure optimizer
adam = Adam(learning_rate=initial_lr)

# 2. Configure training process
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])


## Train the model
1. Load parameters from previous trainings if they exist.
2. Fit the model
3. Save the weights

In [8]:
# Load pretrained model
weights_path = f"weights_{version}.h5" # Name of the file to store the weights
weights_file = Path(weights_path)
weights_load_path = f'{experiment_dir}/{weights_path}'
#if weights_load_path:
#    try:
#        model.load_weights(weights_load_path)
#        print("Loaded model from {}".format(weights_load_path))
#    except:
#        print("Impossible to find weight path. Returning untrained model")

# Fit the model
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=batch_size)

# Save weights
weights_save_path = os.path.join(experiment_rootdir, weights_path)
model.save_weights(weights_save_path)

Epoch 1/30


2024-01-18 11:09:00.660249: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.65GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-18 11:09:11.421949: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.34GiB (rounded to 2516582400)requested by op gradient_tape/model_1/dense_3/MatMul/MatMul_1
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-18 11:09:11.422259: W tensorflow/tsl/framework/bfc_allocator.cc:497] ********************_______________*********************************************************________
2024-01-18 11:09:11.422293: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIR

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_1/dense_3/MatMul/MatMul_1' defined at (most recent call last):
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/tornado/platform/asyncio.py", l

### Plot history for accuracy and loss for the training  process

In [ ]:
plots.accloss(history, modelCNN, experiment_dir, version)

# Testing

### Testing process
Compute the loss function and accuracy for the test data (using `evaluate`)

In [ ]:
# Evaluate model
scores = model.evaluate(test_ds, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss: %.2f" % scores[0])

### Generation of the predictions and the confusion matrix for the test data
1.   Compute predictions with `predict`
2.   Select the most probable class with `argmax`

The confusion matrix will be shown with `plotcm`


In [ ]:

# 1. Get predictions
prob_class = model.predict(test_ds, batch_size=batch_size)

# 2. Prediced labels
y_pred = tf.argmax(prob_class, axis=-1)

# Get ground truth
y_true = tf.argmax(tf.concat([label for image, label in test_ds], axis=0), axis=1)

# Visualize confusion matrix                                           
plotcm.plotcm(experiment_rootdir, version, y_true, y_pred, CLASSES, experiment_rootdir, normalize=True)